## Emerging Market and Inflation HW4

### Chenyu Zhao

## Q1 Compare pricing of the 18m USD-BRL off-shore NDF via 3 approaches using market data provided

In [22]:
import numpy as np

spot = 3.75
multiplier = 10000
tenor = np.array([1, 2])
fwdpoints = np.array([4400,9050])/10000
Z_USD_offshr = np.array([0.9935,0.981])
convert_rate = np.array([0.96,0.4])/100

In [23]:
BRL_252 = np.array([15.33,15.43])/100
USD_linear = np.array([4.03,4.02])/100
Busdays = np.array([341,402,375])
Actualdays = np.array([495,586,547])

In [24]:
## Approach 1

FX = spot + fwdpoints
slope = (FX[1]-FX[0])/(Actualdays[1]-Actualdays[0])

res = slope*(Actualdays[2]-Actualdays[0])+FX[0]
print(res)

4.455714285714286


In [33]:
## Approach 2

USD_offshr = -np.log(Z_USD_offshr)/Actualdays[:-1]*365
res_usd_rate = (USD_offshr[1]-USD_offshr[0])/(Actualdays[1]-Actualdays[0])*(Actualdays[2]-Actualdays[0])+USD_offshr[0]

Z_BRL_onshr = (1+BRL_252)**(-Busdays[:-1]/252)
Z_USD_onshr = (1+USD_linear*Actualdays[:-1]/360)**-1
convert = 1+convert_rate*Actualdays[:-1]/360
Z_BRL_offshr = Z_BRL_onshr*Z_USD_offshr/Z_USD_onshr/convert
BRL_offshr = -np.log(Z_BRL_offshr)/Actualdays[:-1]*365
res_brl_rate = (BRL_offshr[1]-BRL_offshr[0])/(Actualdays[1]-Actualdays[0])*(Actualdays[2]-Actualdays[0])+BRL_offshr[0]

res_Z_usd = np.exp(-res_usd_rate*Actualdays[-1]/365)
res_Z_brl = np.exp(-res_brl_rate*Actualdays[-1]/365)
res = spot*res_Z_usd/res_Z_brl
print(res)

4.417523518987224


In [62]:
## Approach 3

res_usd_offshr_rate = (USD_offshr[1]-USD_offshr[0])/(Actualdays[1]-Actualdays[0])*(Actualdays[2]-Actualdays[0])+USD_offshr[0]
res_usd_onshr_rate = (USD_linear[1]-USD_linear[0])/(USD_offshr[1]-USD_offshr[0])*(res_usd_offshr_rate-USD_offshr[0])+USD_linear[0]
res_convert_rate = (convert_rate[1]-convert_rate[0])/(USD_offshr[1]-USD_offshr[0])*(res_usd_offshr_rate-USD_offshr[0])+convert_rate[0]

PU = 100000/(1+BRL_252)**(Busdays[:-1]/252)
f_CDI = (PU[0]/PU[1])**(252/(Busdays[1]-Busdays[0]))-1
res_PU = PU[0]/(1+f_CDI)**((Busdays[2]-Busdays[0])/252)

res_Z_usd_offshr = np.exp(-res_usd_offshr_rate*Actualdays[-1]/365)
res_convert = 1+res_convert_rate*(Actualdays[-1]/360)
res_Z_brl_onshr = res_PU/100000
res_Z_usd_onshr = 1/(1+res_usd_onshr_rate*Actualdays[-1]/360)
res_Z_brl_offshr = res_Z_brl_onshr/res_convert/res_Z_usd_onshr*res_Z_usd_offshr

res = spot*res_Z_usd_offshr/res_Z_brl_offshr
print(res)

4.415393193371686


## Q2 More economics than math, but suggest effects of upfront fees and minimum tenor requirements as yet another measures of capital control on on-shore USD rates in Brazil

When there is upfront fee, we have 

$$
(r_$^\text{off shr}+s)\cdot \tau=(1-\text{fee rate})\frac{(1+r_$^\text{on shr})^\tau}{1+\hat e_t}-1
$$

and when there is minimum tenor, we have 

$$
(r_$^\text{off shr}+s)\cdot \max(\tau,\tau_\min)=\frac{(1+r_$^\text{on shr})^{\max(\tau,\tau_\min)}}{1+\hat e_t}-1
$$

## Q3 Estimate level of Percentage CDI Swap convexity adjustment for ranges of κ, σ and T. Suggest validity limits where convexity could be safely ignored and where it should be accounted.

according to the lecture,

$$
\text{Cxty}=\exp{(-\frac16\kappa(1-\kappa)\sigma^2T^3)}
$$

In [83]:
def Cxty(k,sigma,T):
    return np.exp(-1/6*k*(1-k)*(sigma**2)*(T**3))

Ts = np.arange(1,11,1)
ks = np.arange(0.5,1.6,0.1)

def Cxty_1(k,T):
    return Cxty(k,0.1*0.15,T)

def Cxty_2(k,T):
    return Cxty(k,0.3*0.15,T)

In [92]:
data_1 = []
for T in Ts:
    temp = []
    for k in ks:
        temp.append(Cxty_1(k,T))
    data_1.append(temp)
    
data_2 = []
for T in Ts:
    temp = []
    for k in ks:
        temp.append(Cxty_2(k,T))
    data_2.append(temp)
        

In [93]:
import pandas as pd

data_1 = pd.DataFrame(data_1,index=Ts,columns=ks)
data_2 = pd.DataFrame(data_2,index=Ts,columns=ks)

In [94]:
# sigma = 0.1
data_1

,0.5,0.6,0.7,0.8,0.9,1.0,1.1,1.2,1.3,1.4,1.5
1,0.999991,0.999991,0.999992,0.999994,0.999997,1.0,1.000004,1.000009,1.000015,1.000021,1.000028
2,0.999925,0.999928,0.999937,0.999952,0.999973,1.0,1.000033,1.000072,1.000117,1.000168,1.000225
3,0.999747,0.999757,0.999787,0.999838,0.999909,1.0,1.000111,1.000243,1.000395,1.000567,1.000760
4,0.999400,0.999424,0.999496,0.999616,0.999784,1.0,1.000264,1.000576,1.000936,1.001345,1.001802
5,0.998829,0.998876,0.999016,0.999250,0.999578,1.0,1.000516,1.001126,1.001830,1.002628,1.003522
6,0.997977,0.998058,0.998300,0.998705,0.999271,1.0,1.000891,1.001946,1.003164,1.004546,1.006093
7,0.996790,0.996918,0.997303,0.997944,0.998843,1.0,1.001416,1.003092,1.005029,1.007229,1.009694
8,0.995212,0.995403,0.995976,0.996933,0.998273,1.0,1.002114,1.004619,1.007516,1.010810,1.014504
9,0.993189,0.993460,0.994276,0.995636,0.997543,1.0,1.003012,1.006583,1.010719,1.015427,1.020715
10,0.990669,0.991040,0.992156,0.994018,0.996631,1.0,1.004134,1.009041,1.014732,1.021222,1.028524


In [95]:
# sigma = 0.3
data_2

,0.5,0.6,0.7,0.8,0.9,1.0,1.1,1.2,1.3,1.4,1.5
1,0.999916,0.999919,0.999929,0.999946,0.999970,1.0,1.000037,1.000081,1.000132,1.000189,1.000253
2,0.999325,0.999352,0.999433,0.999568,0.999757,1.0,1.000297,1.000648,1.001054,1.001513,1.002027
3,0.997724,0.997815,0.998088,0.998543,0.999180,1.0,1.001003,1.002189,1.003560,1.005116,1.006858
4,0.994615,0.994829,0.995474,0.996550,0.998058,1.0,1.002379,1.005197,1.008460,1.012169,1.016332
5,0.989509,0.989926,0.991180,0.993273,0.996210,1.0,1.004651,1.010176,1.016589,1.023906,1.032147
6,0.981940,0.982656,0.984808,0.988404,0.993460,1.0,1.008051,1.017650,1.028839,1.041669,1.056197
7,0.971474,0.972599,0.975983,0.981648,0.989635,1.0,1.012815,1.028173,1.046182,1.066974,1.090702
8,0.957720,0.959376,0.964363,0.972731,0.984568,1.0,1.019190,1.042344,1.069715,1.101605,1.138373
9,0.940344,0.942661,0.949644,0.961399,0.978100,1.0,1.027434,1.060827,1.100709,1.147724,1.202651
10,0.919087,0.922194,0.931578,0.947432,0.970082,1.0,1.037823,1.084371,1.140680,1.208041,1.288044


We can see convexity adjustment is between 0.99 and 1.01 when tenor is less or equal than 3 years, which means with small tenor, convexity can be ignored